In [48]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

# preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

Import the CSV Data as Pandas DataFrame

In [49]:
df = pd.read_csv('../data/data_cleaned.csv')

View the first 5 rows of the dataset

In [50]:
df.head()

,ID,CONSOLE,YEAR,CATEGORY,PUBLISHER,RATING,CRITICS_POINTS,USER_POINTS,SalesInMillions
0,2860,ds,2008,role-playing,Nintendo,E,2.83333,0.30370,1.77926
1,731,wii,2012,simulation,Konami Digital Entertainment,E10+,13.20000,1.64000,0.21505
2,495,pc,2019,shooter,Activision,M,4.56250,0.00641,0.53440
3,2641,ps2,2002,sports,Electronic Arts,E,4.18182,0.32692,1.38396
4,811,ps3,2013,action,Activision,M,2.25926,0.03258,0.08267


In [51]:
print("Categories in 'CONSOLE' variable:   ")
print(df["CONSOLE"].unique())

print("Categories in 'CATEGORY' variable:    ")
print(df["CATEGORY"].unique())

print("Categories in 'PUBLISHER' variable:(FIRST 10)    ")
print(df["PUBLISHER"].unique()[:10])

print("Categories in 'RATING' variable:    ")
print(df["RATING"].unique())

Categories in 'CONSOLE' variable:   
['ds' 'wii' 'pc' 'ps2' 'ps3' 'psp' 'x' 'x360' 'psv' 'gba' 'gc' 'ps' 'wiiu'
 'xone' 'ps4' '3ds' 'dc']
Categories in 'CATEGORY' variable:    
['role-playing' 'simulation' 'shooter' 'sports' 'action' 'platform'
 'strategy' 'racing' 'misc' 'fighting' 'adventure' 'puzzle']
Categories in 'PUBLISHER' variable:(FIRST 10)    
['Nintendo' 'Konami Digital Entertainment' 'Activision' 'Electronic Arts'
 'Sony Computer Entertainment' 'Idea Factory' 'Atari' 'Empire Interactive'
 'Paradox Interactive' 'Lucasarts']
Categories in 'RATING' variable:    
['E' 'E10+' 'M' 'T' 'K-A' 'RP']


#### Preparing X and y variables

In [52]:
X = df.drop("SalesInMillions", axis=1)
y = df["SalesInMillions"]

len(X), len(y)

(3506, 3506)

#### Data Transformation

In [53]:
# create column transformers with different transformers for numeric and categorical variables
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", cat_transformer, cat_features),
        ("StandardScaler", num_transformer, num_features)
    ]
)

#### Split the data into training and testing sets

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

len(X_train), len(y_train), len(X_test), len(y_test)

(2804, 2804, 702, 702)

**Fitting the transformer on the dataset**

In [55]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [56]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

#### Model Training

In [62]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 2.4423
- Mean Absolute Error: 1.2324
- R2 Score: 0.1025
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 11003511871.6583
- Mean Absolute Error: 1991712635.4907
- R2 Score: -26669423290557218816.0000


Lasso
Model performance for Training set
- Root Mean Squared Error: 2.5780
- Mean Absolute Error: 1.2521
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 2.1318
- Mean Absolute Error: 1.1705
- R2 Score: -0.0011


Ridge
Model performance for Training set
- Root Mean Squared Error: 2.4448
- Mean Absolute Error: 1.2456
- R2 Score: 0.1007
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 2.0246
- Mean Absolute Error: 1.2211
- R2 Score: 0.0971


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 2.0401
- Mean Absolute Error: 1.0802
- 

#### Results

In [63]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [64]:
results = pd.DataFrame(zip(model_list, r2_list), columns=["Model Name", "R2 Score"]).sort_values(by="R2 Score", ascending=False)
results

,Model Name,R2 Score
7,CatBoosting Regressor,0.53651
6,XGBRegressor,0.42050
3,K-Neighbors Regressor,0.40082
5,Random Forest Regressor,0.32833
2,Ridge,0.09709
4,Decision Tree,0.06096
1,Lasso,-0.00106
8,AdaBoost Regressor,-0.14162
0,Linear Regression,-26669423290557218816.00000


**Best Performing model is CatBoost**

In [60]:
cat_boost = CatBoostRegressor(verbose=False)
catboost = cat_boost.fit(X_train, y_train)
y_pred = cat_boost.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 53.65


**Difference between Actual and Predicted Values**

In [61]:
pred_df=pd.DataFrame({'Actual Value':y_test,'Predicted Value':y_pred,'Difference':y_test-y_pred})
pred_df

,Actual Value,Predicted Value,Difference
3272,4.06308,2.90685,1.15624
315,2.07026,2.10746,-0.03720
2179,4.15179,11.97386,-7.82207
3359,1.87589,2.19467,-0.31878
430,0.07621,1.85242,-1.77622
...,...,...,...
227,2.28710,2.12104,0.16606
555,1.92877,1.78984,0.13893
729,0.16404,2.11292,-1.94888
2656,1.84495,2.01335,-0.16841
